In [5]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="vikky_workspace")
exp = Experiment(workspace=ws, name="vikky_workspace")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: vikky_workspace
Azure region: eastus
Subscription id: 784d8740-4c70-4d20-8f45-b2bfecf94029
Resource group: udacityprojects


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_target_name = "vikkycompute"
compute_cluster = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes = 4)
my_target =  ComputeTarget.create(ws, compute_target_name, compute_cluster)
### YOUR CODE HERE ###

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal,choice
import os
import shutil
# Specify parameter sampler
ps = RandomParameterSampling({
    "learning_rate": normal(10, 3),
    "keep_probability": uniform(0.05, 0.1),
    "Regularization Strength:": (1.0),
    "Max iterations:": (1000),
    "Accuracy:": choice(0.906, 0.908, 0.910)

}) 

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor=0.4)
### YOUR CODE HERE ###
#os.listdir()
if "training" not in os.listdir():
    os.mkdir("training")
    print("Path created")
    shutil.copy("train.py","./training")
    print('copied')
#copy training file
#else:
    #shutil.copy("train.py","./training")
    #print('copied')

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = 'training/',
                 entry_script = 'train.py',
                 #script_params = { "Regularization Strength:": 1.0, "Max iterations:": 1000, "Accuracy:":1.0},
                 compute_target = 'vikkycompute')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)
### YOUR CODE HERE ###

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=best_run.id, filename='outputs/best_run_model.joblib')
print('model saved')
### YOUR CODE HERE ###

Run(Experiment: vikky_workspace,
Id: HD_47fcb5fc-7925-4271-b46f-3a53d2a315f3_2,
Type: azureml.scriptrun,
Status: Completed)
model saved


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dat = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
df = TabularDatasetFactory.from_delimited_files(dat)
print(df) ### YOUR CODE HERE ###

TabularDataset
{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [12]:
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn import datasets
from azureml.core.dataset import Dataset
import numpy as np
from train import clean_data
#from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn import preprocessing


x, y = clean_data(df)
x_train, x_test, y_train, y_test = train_test_split(x, y)
training_data = pd.concat([x_train, y_train], axis = 1)

if not os.path.isdir('dataset'):
    os.mkdir('dataset')


training_data.to_csv('dataset/y.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./dataset', target_path='ydataset', overwrite=True, show_progress=True)

training_data = Dataset.Tabular.from_delimited_files(path=ds.path('ydataset/y.csv'))



Uploading an estimated of 1 files
Uploading ./dataset/y.csv
Uploaded ./dataset/y.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [26]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target = "vikkycompute",
    task="classification",
    primary_metric="accuracy",
    training_data= training_data,
    label_column_name= "y",
    n_cross_validations=5)

In [27]:
# Submit your automl run
experiment = Experiment(ws, 'vikkycompute')
print(experiment)
auto_run = experiment.submit(automl_config, show_output = True)
auto_run
### YOUR CODE HERE ###

Experiment(Name: vikkycompute,
Workspace: vikky_workspace)
Running on remote.
Running on remote compute: vikkycompute
Parent Run ID: AutoML_2c207886-ff6b-4fb9-a37c-97762a0f454a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+------------

Experiment,Id,Type,Status,Details Page,Docs Page
vikkycompute,AutoML_2c207886-ff6b-4fb9-a37c-97762a0f454a,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [29]:
# Retrieve and save your best automl model.
best_run, best_model = auto_run.get_output()
best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')
### YOUR CODE HERE ###

Model(workspace=Workspace.create(name='vikky_workspace', subscription_id='784d8740-4c70-4d20-8f45-b2bfecf94029', resource_group='udacityprojects'), name=automl_best_model.pkl, id=automl_best_model.pkl:1, version=1, tags={}, properties={})